In [ ]:
import pandas as pd

df = pd.read_excel("data.xlsx")

df

,original,modern
0,Riddling confession finds but riddling shrift .,A jumbled confession can only receive a jumble...
1,Then plainly know my heart's dear love is set ...,I love rich Capulet's daughter .
2,"As mine on hers , so hers is set on mine , And...",We're bound to each other in every possible wa...
3,"When and where and how We met , we wooed and m...",I'll tell you more later about when and where ...
4,"Holy Saint Francis , what a change is here !","Holy Saint Francis , this is a drastic change !"
...,...,...
21070,"How now , my metal of India ?","How are you , my golden girl ?"
21071,Get you all three into the boxtree .,"Hide behind the boxwood hedge , all three of y..."
21072,Malvolio's coming down this walk .,Malvolio's coming down the path .
21073,He has been yonder i ' the sun practising beha...,He's been over there practicing how to act for...


In [ ]:
df['combined'] = '<s>' + df.modern + '</s>' + '>>>>' + '<p>' + df.original + '</p>'
df['combined'] = df.combined.to_csv('combined.txt', sep='\n', index=False)

In [ ]:
from transformers import (AutoModelWithLMHead,
                          AutoConfig, 
                          Trainer, 
                          AutoTokenizer, 
                          TextDataset, 
                          DataCollatorForLanguageModeling, 
                          TrainingArguments)
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 25.3 MB/s 
     |████████████████████████████████| 7.6 MB 64.5 MB/s 
     |████████████████████████████████| 182 kB 73.4 MB/s 


In [ ]:
def modelTrainer(text_path, epochs, model='gpt2', batch_size=8, cache_dir='cache'):

  model = AutoModelWithLMHead.from_pretrained(model)
  tokenizer = AutoTokenizer.from_pretrained('gpt2')
  data_collator = DataCollatorForLanguageModeling (tokenizer=tokenizer, mlm=False)
  train_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=text_path,
      block_size=256
  )
  training_args = TrainingArguments(
      output_dir = "gpt2_fine_tune/{}".format(os.path. basename (text_path)),
      num_train_epochs = epochs, 
      per_device_train_batch_size = batch_size, 
      warmup_steps = 500, 
      save_steps = 2000,
      logging_steps = 10
  )
  trainer = Trainer(
      model = model,
      args = training_args, 
      data_collator = data_collator, 
      train_dataset = train_dataset
  )
  trainer.train()
  trainer.save_model()

In [ ]:
modelTrainer('combined.txt', epochs = 20)

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (798480 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3119
  Num Epochs = 20
  Instantaneous batch size per device = 8
  

Step,Training Loss
10,3.266400
20,3.120100
30,3.054300
40,2.856800
50,2.877000
60,2.751800
70,2.841400
80,2.651000
90,2.528900
100,2.610800


Saving model checkpoint to gpt2_fine_tune/combined.txt/checkpoint-2000
Configuration saved in gpt2_fine_tune/combined.txt/checkpoint-2000/config.json
Model weights saved in gpt2_fine_tune/combined.txt/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to gpt2_fine_tune/combined.txt/checkpoint-4000
Configuration saved in gpt2_fine_tune/combined.txt/checkpoint-4000/config.json
Model weights saved in gpt2_fine_tune/combined.txt/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to gpt2_fine_tune/combined.txt/checkpoint-6000
Configuration saved in gpt2_fine_tune/combined.txt/checkpoint-6000/config.json
Model weights saved in gpt2_fine_tune/combined.txt/checkpoint-6000/pytorch_model.bin


Step,Training Loss
10,3.266400
20,3.120100
30,3.054300
40,2.856800
50,2.877000
60,2.751800
70,2.841400
80,2.651000
90,2.528900
100,2.610800




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to gpt2_fine_tune/combined.txt
Configuration saved in gpt2_fine_tune/combined.txt/config.json
Model weights saved in gpt2_fine_tune/combined.txt/pytorch_model.bin


In [ ]:
from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("gpt2_fine_tune/combined.txt")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1132: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file gpt2_fine_tune/combined.txt/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2_fine_tune/combined.txt",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_at

In [ ]:
def split(txt, seps):
    default_sep = seps[0]

    # we skip seps[0] because that's the default separator
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]
    
def paraphrase(input_sentence):
  p = generator('<s>'+input_sentence+'</s>>>>><p>')
  return split(p[0]['generated_text'].split('</s>>>>><p>')[1].split('</p>')[0], ('/s','<', '>'))[0]

In [ ]:
paraphrase('This is awesome!')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'It is an absolute wonder!'

In [ ]:
paraphrase('Don’t be so childish')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Thou art not a child.'

In [ ]:
paraphrase('Sorry to hear that.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"I must repent what I've heard."

In [ ]:
paraphrase('I’ll be there at that time.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'I shall be there presently., Now, my gracious lord, Would you please to go away and attend my brother?'